<a href="https://colab.research.google.com/github/kmt19/siim_otani/blob/master/drive2gcp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google import colab
colab.drive.mount('/content/gdrive')
!ls 'gdrive/My Drive'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'    dicom-images-train   train_img.csv
 dicom-images-test   train.csv		  手法のアイデア.gdoc


In [18]:
!pip install pydicom
!git clone https://github.com/kmt19/siim_otani/
!ls

fatal: destination path 'siim_otani' already exists and is not an empty directory.
gdrive	sample_data  siim_otani


In [19]:
import os
import pydicom
from glob import glob
from tqdm import tqdm
import pandas as pd
#from siim_otani.mask_functions import mask2rle,rle2mask
#from siim_otani.pipline2 import get_dataloaders
DRIVE_DIR = 'gdrive/My Drive/'
folders = os.listdir(DRIVE_DIR)
print(folders)


['手法のアイデア.gdoc', 'dicom-images-test', 'dicom-images-train', 'Colab Notebooks', 'train.csv', 'train_img.csv']


In [20]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
print(os.listdir(train_dicom_dir)[0])
a_dir = os.path.join(train_dicom_dir, os.listdir(train_dicom_dir)[0])
print(os.listdir(a_dir))
b_dir = os.path.join(a_dir, os.listdir(a_dir)[0])
print(os.listdir(b_dir))
"""
ここから各Directory内のlistを読み込み、
BASE_DIR = 'train_dicom_dir/%s/%s/' % (file_list, file_list2)を
for文でもらってきて最終的にpathを保存したdataframeを作成する。
"""
for filename in os.listdir(train_dicom_dir):
  BASE_DIR = 'train_dicom_dir/%s/%s/' % (filename, filename)


1.2.276.0.7230010.3.1.2.8323329.13318.1517875244.629616
['1.2.276.0.7230010.3.1.3.8323329.13318.1517875244.629615']
['1.2.276.0.7230010.3.1.4.8323329.13318.1517875244.629617.dcm']


In [0]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import pil_loader
import pydicom
from PIL import Image
from __future__ import print_function
from collections import defaultdict, deque
import datetime
import pickle
import time
import torch.distributed as dist
import errno
import collections
import numpy as np
import torch.utils.data
from torchvision import transforms
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
#import torch_xla

In [0]:
def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)

def load_mask(self, height, width, annotations):
    count = len(annotations)
    if count == 0 or (count == 1 and annotations[0] == -1): # empty annotation
        mask = np.zeros((height, weight, 1), dtype=np.uint8)
        class_ids = np.zeros((1,), dtype=np.int32)
    else:
        mask = np.zeros((height, width, count), dtype=np.uint8)
        class_ids = np.zeros((1,), dtype=np.int32)
        mask[:, :, i] = rle2mask(annotations, height, width).T
    return mask.astype(np.bool), class_ids.astype(np.int32)
  
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets.folder import pil_loader
import pydicom
from PIL import Image

class SIIMDataset(Dataset):
    """
    args:
    - phase    : phase of the dataset used (∈{"train", "test"})
    - transform:  
    """
    def __init__(self, phase, df, transform=None):
        self.phase = phase
        #self.root_dir = root_dir
        self.cols = 2 if phase == "train" else 1
        self.df = df #pd.read_csv(f"{root_dir}/{phase}.csv", usecols = [i for i in range(self.cols)])
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx][0]
        image = pydicom.read_file(img_path).pixel_array
        shape = np.array(image).shape
        image = Image.fromarray(image)
        label = None
        if self.phase == "train":
            label = self.df.loc[idx][1]
            label = load_mask(shape[0],shape[1],label)
        image = image.convert("RGB")

        if self.transform:
            image = self.transform(image)   # transform image to tensor
            if self.phase == "train":
                label = self.transform(label)
        #image = torch.stack(image)
        sample = {'image':image, 'label':label}
        return sample

def get_dataloaders(data, batch_size=8, study_level=False):
    '''
    Returns dataloader pipeline with data augmentation
    '''
    data_transforms = {
        'train': transforms.Compose([
                transforms.Resize((224, 224)),
                #transforms.RandomHorizontalFlip(),
                #transforms.RandomRotation(10),
                transforms.ToTensor(),
                #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
        ]),
        'test': transforms.Compose([
            #transforms.Resize((224, 224)),
            transforms.ToTensor(),
            #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    phase=["train","test"]
    image_datasets = {x:SIIMDataset(x,data[x],transform=data_transforms[x]) for x in phase}
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in phase}
    return dataloaders


In [26]:
train_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-train')
test_dicom_dir = os.path.join(DRIVE_DIR, 'dicom-images-test')
data_files={}
phase = ["train","test"]
data_files["train"] = pd.DataFrame(columns=['Path', 'ID'])
data_files["test"] = pd.DataFrame(columns=['Path'])
SEGMENTATION = DRIVE_DIR + 'train.csv'
anns = pd.read_csv(SEGMENTATION)
train_list=os.listdir(train_dicom_dir)

for i, filename in tqdm(enumerate(train_list)):
  train_glob = f'{train_dicom_dir}/{filename}/*/*.dcm'
  data_files["train"].loc[i] = [glob(train_glob)[0], anns.loc[i][1]]
  if i==10:
    break

test_list=os.listdir(test_dicom_dir)  
for i, filename in tqdm(enumerate(test_list)):
  test_glob = f'{test_dicom_dir}/{filename}/*/*.dcm'
  data_files["test"].loc[i] = [glob(test_glob)[0]]
  if i==10:
    break
traindataloader = get_dataloaders(data_files)


0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [0]:
def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict
  
def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha
    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)

def train_one_epoch(model, optimizer, data_loader, epoch, print_freq):
    model.train()
    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in data_loader:
        images = list(torch.jit.trace(image) for image in images)
        targets = [{k: torch.jit.trace(v) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

In [34]:
# create mask rcnn model
num_classes = 2
#device = torch.device('cuda:0')
import torchvision
model_ft = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model_ft.roi_heads.box_predictor.cls_score.in_features
model_ft.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features_mask = model_ft.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model_ft.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
for param in model_ft.parameters():
    param.requires_grad = True
params = [p for p in model_ft.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.1)
traced_net = torch.jit.trace(model_ft, (torch.rand(1, 3, 224, 224),torch.rand(1,1,224, 224)))
num_epochs = 6
for epoch in range(num_epochs):
    train_one_epoch(traced_net, optimizer, traindataloader["train"], epoch, print_freq=100)
    lr_scheduler.step()

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:435: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  'incorrect results).', category=RuntimeWarning)


IndexError: ignored